# Deep DeePC Main Code


## 1. Deep DeePC Class

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import scipy

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F

from tqdm import tqdm


In [6]:
class GreenhouseParams:
    def __init__(self):
        self.params = {}
        self._initialize_params()

    def _initialize_params(self):
        # Control parameters
        self.params["nx"] = 5  # Number of state variables
        self.params["nu"] = 7  # Number of control inputs
        self.params["nd"] = 3  # Number of disturbances (e.g., outdoor weather)
        self.params["N_v"] = 2  # VAR Order for weather prediction model

        # Cost function parameters
        self.params["cc_e"] = 0.2051             # Electricity cost [$/kWh]
        self.params["cc_h"] = 1.43e-2             # Water cost [$/kg]
        self.params["cc_c"] = 2e-1                # CO2 cost [$/kg]   [Chen2022appeng]
        self.params["cc_dw"] = 4.44444            # Dry weight price of lettuce [$/kg]
        self.params["c_dehum_eev"] = 3.0         # Dehumidification cost [kg / kWh]
        
        self.params["q1"] = 5e+1                  # Weight for state tracking in cost function
        self.params["q2"] = 1e+0                  # Regularization weight for control inputs
        self.params["q3"] = 1e+2                  # Regularization weight for crop dry weight
        
        # Hyperparameters for control inputs
        self.params["u_min"] = 0  # 
        self.params["u_max"] = 1  #
        self.params["du_min"] = -0.05 # [Svensen2024]
        self.params["du_max"] = 0.05  #
        
        self.params["u_fan_max"] = 1e-2   # Max fan speed [m/s] [Benjamin2024natfood]
        self.params["u_dehum_max"] = 2e-5   # Max fan speed [kg/m²/s] [Benjamin2024natfood]
        self.params["u_pad_max"] = 5e-3 # maximum airflow rate through the pad [m/s] (Considering that 0.005 [m/s] * A_v ())
        
        self.params["phi_co2_max"] = 4e-6 # 4.6e-6 # Previous value: 2e-2 # Max CO2 injection rate [kg/m²/s] [Benjamin2024natfood]
        self.params["P_heat"] = 100       # Heating power for direct air heater [W/m²]
        self.params["c_COP"] = 3          # coefficient of performance 
        self.params["P_light"] = 100      # Lighting power [W/m²]
        self.params["P_fan"] = 20         # Fan system power [W/m²]
        self.params["P_fog"] = 10         # Fogging system power [W/m²]
        self.params["C_out"] = 400        # Outdoor CO2 concentration [ppm]
        
        self.params["c_lamp"] = 0.52    # Efficiency of the lamp heating system [Graamans2020appeng], [Benjamin2024natfood]
        self.params["fan_eff"] = 1000 * 0.00047194745 * 15 # 7.07921175 [m³/s/kW]
        self.params["pad_eff"] = 10     # Cooling pad efficiency [m³/s/kW]: usually 7 ~ 15
        
        # Cooling Pad Coefficients
        self.params["eta_p"] = 0.8       # pad efficiency [-]
        
        # Environmental constraints for daytime and nighttime
        self.params["Ti_min_day"] = 22        # Min daytime temperature [°C]
        self.params["Ti_max_day"] = 28        # Max daytime temperature [°C]
        self.params["RHi_min_day"] = 60       # Min daytime humidity [%]
        self.params["RHi_max_day"] = 85       # Max daytime humidity [%]
        self.params["Ci_min_day_ppm"] = 1000  # Min daytime CO2 concentration [ppm]
        self.params["Ci_max_day_ppm"] = 1400  # Min daytime CO2 concentration [ppm]
        self.params["Dw_min"] = 0             # Min Crop dry weight [kg/m²]

        self.params["Ti_min_night"] = 18      # Min nighttime temperature [°C]
        self.params["Ti_max_night"] = 24      # Max nighttime temperature [°C]
        
        self.params["RHi_min_night"] = 50     # Min nighttime humidity [%]
        self.params["RHi_max_night"] = 75     # Max nighttime humidity [%]
        
        self.params["Ci_min_night_ppm"] = 400  # Min daytime CO2 concentration [ppm]
        self.params["Ci_max_night_ppm"] = 1000 # Min daytime CO2 concentration [ppm]
        self.params["Dw_max"] = 1e1         # Max Crop dry weight [kg/m²]

        # Greenhouse construction parameters
        self.params["w"] = 10.4  # Greenhouse width [m]
        self.params["l"] = 6.4   # Greenhouse length [m]
        self.params["h"] = 5     # Greenhouse height [m]
        self.params["V"] = self.params["w"] * self.params["l"] * self.params["h"]  # Greenhouse volume [m³]
        
        self.params["r_veg"] = 0.8   # Vegetation area ratio
        self.params["A_c"] = (np.pi * self.params["l"] * self.params["w"] / 2) + 2 * (self.params["w"] * self.params["h"])  # Cover area [m²] (Assume: Single-span semi-cylindrical (Quonset) Greenhouse)
        self.params["A_s"] = self.params["w"] * self.params["l"]        # Surface area [m²]
        self.params["A_v"] = self.params["A_s"] * self.params["r_veg"]  # Vegetation area [m²]
        self.params["A_p"] = 5                                          # Cooling pad area [m²]

        # Heat transfer equation [Lin2021appeng], [Benjamin2024natfood]
        self.params["T_abs"] = 273.15     # Absolute temperature [°C]
        self.params["L_v"] = 2256.4       # Latent heat of evaporation [J/g]
        self.params["k_cap_q"] = 3e4    # Heat capacity [J/m²/°C]
        self.params["k_cap_q_v"] = 1290 # Heat transfer coefficient [W/m²/°C]
        self.params["k_cap_h"] = (self.params["V"] / self.params["A_v"]) # Mass capacity for humidity [m]
        self.params["k_cap_c"] = (self.params["V"] / self.params["A_v"]) # Mass capacity for CO2 [m]
        
        self.params["rho_i"] = 1.225    # Internal air density [kg/m³]
        self.params["c_i"] = 1000       # Heat capacity of internal air [J/kg/°C]
        self.params["alpha1"] = 0.7     # transmission coefficient of cover material [-]
        self.params["c_cov"] = 0.3 # heat transfer coefficient of cover material [W/m²/°C] [Benjamin2024natfood]

        # coefficient of pad cooling equation [J/m³/K]
        self.params["K_p"] = (self.params["A_v"] / self.params["A_p"]) * self.params["rho_i"] * self.params["c_i"]
        
        # Mass balance equation
        self.params["p_gc"] = 1.8e-3  # Mass balance parameter [m*C^(-1/3)/s]
        self.params["m_co2"] = 0.04401  # Molar mass of CO2 [mg/µmol]
        self.params["m_carb"] = 30e-3    # Molar mass of carbohydrates [mg/µmol]
        self.params["r_b"] = 150         # Boundary layer resistance [s/m]
        self.params["gamma"] = 8e-3      # CO2 compensation point
        
        self.params["c_a_pl"] = 62.8 # [m^2/kg]
        self.params["c_v_pl_ai"] = 3.6E-3 # mass transfer coefficient [m/s] [Stanghellini1987]
        self.params["c_v_0"] = 0.85 # Calibration parameter [-]
        self.params["c_v_1"] = 611 # [J/m^3] [Goudriaan1977]
        self.params["c_v_2"] = 17.4 # [-] [Goudriaan1977]
        self.params["c_v_3"] = 239 # [°C] [Goudriaan1977]
        self.params["mw_water"] = 18 # molecular mass of water [kg/kmol]
        self.params["c_R"] = 8314 # gas constant [J/K/kmol]
        
        # Two-state crop model for lettuce (Van Hanten)
        self.params["c_alpha"] = 0.68 # physical  [-]
        self.params["c_beta"] = 0.8
        self.params["c_co2_1"] = 5.11e-6
        self.params["c_co2_2"] = 2.30e-6
        self.params["c_co2_3"] = 6.29e-4
        self.params["c_gamma"] = 5.2e-5
        
        self.params["c1"] = 3.55e-9
        self.params["c_lar_d"] = 62.5e-3
        self.params["c_lar_s"] = 75 # 75E-3? shoot structural leaf area ratio [1/m^2/kg]
        self.params["c_tau"] = 0.07
        self.params["c_k"] = 0.9
        self.params["c_bnd"] = 0.004
        self.params["c_pld"] = 53
        
        self.params["c_r_gr_max"] = 5E-6 # [1/s]
        self.params["c_Q10_gr"] = 1.6 # Q10-factor for the maintenance respiration [-]
        self.params["c_Q10_resp"] = 2.0 # Q10-factor for the maintenance respiration [-]
        self.params["c_Q10_Gamma"] = 2 # [-]
        self.params["c_resp_s"] = 3.47E-7 # maintenance respiration rates for the shoot [1/s]
        self.params["c_resp_r"] = 1.16E-7 # maintenance respiration rates for the root [1/s]
        self.params["c_par"] = 1 # [-]
        self.params["c_rad_rf"] = 1 # [-]
        self.params["c_eps"] = 17E-9 # light use efficiency at very high CO2 [kg/J] [Goudriaan1985]
        self.params["c_Gamma"] = 7.32E-5 # [kg/m^3] [Goudriaan1985]
        self.params["c_car_1"] = -1.32E-5 # carboxylation resistance fitting parameters 1 [m/s/C^2] [Goudriaan1987]
        self.params["c_car_2"] =  5.94E-4 # carboxylation resistance fitting parameters 2 [m/s/C] [Goudriaan1987]
        self.params["c_car_3"] = -2.64E-3 # carboxylation resistance fitting parameters 3 [m/s] [Goudriaan1987]
        
        self.params["c_bnd"] = 0.004 # boundary layer conductance [m/s]
        self.params["c_stm"] = 0.007 # stomatal conductance [m/s]
        self.params["c_tau"] = 0.07 # ratio of the root dry weight to the total crop dry weight (measurement)
        self.params["c_k"] = 0.9 #
        
        self.params["Hi_min_day"] = (self.params["RHi_min_day"] / 100) * ( (1.0272 * self.params["Ti_min_day"] - 1.8959) * 1E-3 )   # Min daytime absolute humidity [kg/m^3]
        self.params["Hi_max_day"] = (self.params["RHi_max_day"] / 100) * ( (1.0272 * self.params["Ti_max_day"] - 1.8959) * 1E-3 )   # Min daytime absolute humidity [kg/m^3]
        self.params["Hi_min_night"] = (self.params["RHi_min_night"] / 100) * ( (1.0272 * self.params["Ti_min_night"] - 1.8959) * 1E-3 )   # Min daytime absolute humidity [kg/m^3]
        self.params["Hi_max_night"] = (self.params["RHi_max_night"] / 100) * ( (1.0272 * self.params["Ti_max_night"] - 1.8959) * 1E-3 )   # Min daytime absolute humidity [kg/m^3]
        
        self.params["Ci_min_day"] = self.params["Ci_min_day_ppm"] * (self.params["m_co2"]/24.45) * 1E-3     # Min daytime CO2 [kg/m³]
        self.params["Ci_max_day"] = self.params["Ci_max_day_ppm"] * (self.params["m_co2"]/24.45) * 1E-3     # Max daytime CO2 [kg/m³]
        self.params["Ci_min_night"] = self.params["Ci_min_night_ppm"] * (self.params["m_co2"]/24.45) * 1E-3 # Min nighttime CO2 [kg/m³]
        self.params["Ci_max_night"] = self.params["Ci_max_night_ppm"] * (self.params["m_co2"]/24.45) * 1E-3 # Max nighttime CO2 [kg/m³]
        
        
    def get_param(self, key):
        return self.params.get(key, None)

    def set_param(self, key, value):
        self.params[key] = value
        
    def set_initial(self):
        self.params["Ti_0"] = 20.0                       # unit: [°C]
        self.params["RHi_0"] = 60                        # unit: [%]
        
        self.params["Hi_sat_0"] = (1.0272 * self.params["Ti_0"] - 1.8959) * 1E-3
        self.params["Hi_0"] = (self.params["RHi_0"] / 100) * self.params["Hi_sat_0"] # unit: [kg/m^3]
        self.params["Ci_0"] = 400.0 * (0.04401 / 24.45) * 1E-3  # unit change: [ppm] -> [kg/m³]
        self.params["Dn_0"] = 0.72E-3 * 0.25 # [kg/m^2] # Zhang and Kacira (2020)
        self.params["Ds_0"] = 0.72E-3 * 0.75 # [kg/m^2] # Zhang and Kacira (2020)
        
        x0 = np.array([self.params["Ti_0"], self.params["Hi_0"], self.params["Ci_0"], self.params["Dn_0"], self.params["Ds_0"]], dtype=float)
        u0 = np.array(np.zeros((self.params["nu"], 1)))
        
        return x0, u0
    
    def set_min_max(self):
        x_min_day = np.array([self.params["Ti_min_day"], self.params["Hi_min_day"], self.params["Ci_min_day"], self.params["Dw_min"], self.params["Dw_min"]], dtype=float)
        x_max_day = np.array([self.params["Ti_max_day"], self.params["Hi_max_day"], self.params["Ci_max_day"], self.params["Dw_max"], self.params["Dw_max"]], dtype=float)
        
        x_min_night = np.array([self.params["Ti_min_night"], self.params["Hi_min_night"], self.params["Ci_min_night"], self.params["Dw_min"], self.params["Dw_min"]], dtype=float)
        x_max_night = np.array([self.params["Ti_max_night"], self.params["Hi_max_night"], self.params["Ci_max_night"], self.params["Dw_max"], self.params["Dw_max"]], dtype=float)
        
        return x_min_day, x_max_day, x_min_night, x_max_night
    

In [7]:
class CropClimateModel:
    def __init__(self, greenhouse_params):
        self.p = greenhouse_params  # So we can access p.get_param("rho_i"), etc.

    def cc_model_gh(self, t, x, u, o):
        # Unpack states: Dn (Non-structural dry weight), Ds (Structural dry weight)
        Ti, Hi, Ci, Dn, Ds = x # Dn, Ds unit: [kg/m²]
        
        # Unpack controls
        u_heat, u_pad, u_blind, u_light, u_fan, u_dehum, u_co2 = u

        # Unpack outdoor disturbances
        To, Ho, Io = o
        Co = self.p.get_param("C_out") * (0.04401 / 24.45) * 1E-3 # unit change: [ppm] -> [kg/m³]
        
        # Retrieve parameters from GreenhouseParams
        w_sigma     = self.p.get_param("w_sigma")  # noise covariance (3x3)
        
        k_cap_q = self.p.get_param("k_cap_q")
        k_cap_q_v = self.p.get_param("k_cap_q_v")
        k_cap_h = self.p.get_param("k_cap_h")
        k_cap_c = self.p.get_param("k_cap_c")
        
        T_abs       = self.p.get_param("T_abs")  # 
        L_v         = self.p.get_param("L_v")    # latent heat of evaporation [J/g]
        alpha1      = self.p.get_param("alpha1")
        c_cov       = self.p.get_param("c_cov")
        A_c         = self.p.get_param("A_c")
        A_v         = self.p.get_param("A_v")
        
        P_light     = self.p.get_param("P_light")
        c_lamp      = self.p.get_param("c_lamp")
        phi_co2_max = self.p.get_param("phi_co2_max")
        u_fan_max   = self.p.get_param("u_fan_max")
        u_dehum_max = self.p.get_param("u_dehum_max")
        u_pad_max   = self.p.get_param("u_pad_max") 
        # u_fog_max   = self.p.get_param("u_fog_max")
        P_heat      = self.p.get_param("P_heat")
        p_gc        = self.p.get_param("p_gc")
        
        # Mass balance equation parameters (Crop canopy transpiration)
        c_a_pl = self.p.get_param("c_a_pl")
        c_v_pl_ai = self.p.get_param("c_v_pl_ai")
        c_v_0 = self.p.get_param("c_v_0")
        c_v_1 = self.p.get_param("c_v_1")
        c_v_2 = self.p.get_param("c_v_2")
        c_v_3 = self.p.get_param("c_v_3")
        mw_water = self.p.get_param("mw_water")
        c_R = self.p.get_param("c_R")
        
        # Two state Crop model parameters
        c_alpha = self.p.get_param("c_alpha")
        c_beta  = self.p.get_param("c_beta")
        c_r_gr_max = self.p.get_param("c_r_gr_max")
        c_Q10_gr = self.p.get_param("c_Q10_gr")
        c_Q10_resp = self.p.get_param("c_Q10_resp")
        c_Q10_Gamma   = self.p.get_param("c_Q10_Gamma")
        
        c_resp_s = self.p.get_param("c_resp_s")
        c_resp_r = self.p.get_param("c_resp_r")
        c_par = self.p.get_param("c_par")
        c_rad_rf = self.p.get_param("c_rad_rf")
        c_lar_s = self.p.get_param("c_lar_s")
        c_eps   = self.p.get_param("c_eps")
        c_Gamma   = self.p.get_param("c_Gamma")
        
        c_car_1   = self.p.get_param("c_car_1")
        c_car_2   = self.p.get_param("c_car_2")
        c_car_3   = self.p.get_param("c_car_3")
        
        c_bnd   = self.p.get_param("c_bnd")
        c_stm   = self.p.get_param("c_stm")
        
        c_tau   = self.p.get_param("c_tau")
        c_k     = self.p.get_param("c_k")
        
        eta_p = self.p.get_param("eta_p")
        K_p = self.p.get_param("K_p")
        
        # Generate process noise, akin to MATLAB's mvnrnd(zeros, w_sigma)
        # w = np.random.multivariate_normal(mean=np.zeros(nx), cov=w_sigma)

        # 1) Heat transfer
        Q_sol  = alpha1 * (1 - u_blind) * Io * 1E-1
        Q_lamp = (1 - c_lamp) * P_light * u_light
        Q_heat = P_heat * u_heat
        
        # 1-1) Heat flux via cooling pad
        Ho_sat = (1.0272 * To - 1.8959) * 1E-3 # [Hu2023appeng]
        RHo = (Ho / Ho_sat) * 100 # outdoor relative humidity [%]
        T_wb = self.T_wb_calculation(To, RHo)             # Wet bulb temperature 
        Tp = To - self.p.get_param("eta_p") * (To - T_wb) # Cooling pad temperature
        Q_cool = K_p * u_pad_max * (Tp - Ti) * u_pad      # heat flux by cooling pad [W/m²]
        
        # 1-2) Total light intensity to canopy
        # I_total = Q_sol + (c_lamp * P_light * u_light) # [W/m^2]
        I_total = alpha1 * (1 - u_blind) * Io + (c_lamp * P_light * u_light) # [W/m^2]

        # 2) Humidity saturation
        # Hi_sat = ((c_v_1 * mw_water)/(c_R * (Ti + T_abs))) * np.exp((c_v_2 * Ti)/(Ti + c_v_3)) # [kg/m^3]
        Hi_sat = (1.0272 * Ti - 1.8959) * 1e-3 # [kg/m^3]
        
        # 3) Canopy Transpiration (Heat, Mass)
        H_transp = (1 - np.exp(-c_a_pl * Ds)) * c_v_pl_ai * ( (c_v_0 * Hi_sat) - Hi ) # [kg/m^2/s]
        Q_trans = L_v * 1e3 * H_transp

        # 4) Ventilation (Heat, Mass)
        # Q_vent = (1 / self.p.get_param("A_s")) * rho_i * c_i * (Ti - To) * u_fan_max * u_fan
        Q_vent = k_cap_q_v * (Ti - To) * u_fan_max * u_fan
        H_vent  = (Hi - Ho) * u_fan_max * u_fan

        # 4-1) Dehumidification
        H_dehum = u_dehum_max * u_dehum # [kg/m²/s]
        
        # 5) Cover effect (Heat, Mass)
        if Ti <= To:
            g_c = 0
        else:
            g_c = p_gc * (Ti - To) ** (1 / 3)

        Q_cov  = c_cov * (A_c/A_v) * (Ti - To) # [Benjamin2024natfood]
        H_cov   = g_c * (0.2522 * np.exp(0.0485 * Ti) * 1E-3 * (Ti - To) - (Hi_sat - Hi)) # [kg/m^3]

        # 6-1) Single-state Crop Model (Van Henten)s
        # Dw_numerator = c1*I_total * (-c_co2_1*Ti ** 2 + c_co2_2*Ti - c_co2_3) * (Ci - c_gamma)
        # DW_denominator = c1*I_total + (-c_co2_1*Ti ** 2 + c_co2_2*Ti - c_co2_3) * (Ci - c_gamma)
        # dDw_dt = (c_alpha * c_beta) * (1-np.exp(-c_pld*Dw)) * (Dw_numerator / DW_denominator) - c_resp1 * Dw * 2 ** (0.1*(Ti-25))
        
        # 6-2) Two-state Crop Model (Van Henten, 1994)
        Gamma = c_Gamma * c_Q10_Gamma ** ((Ti - 20)/10) # 
        eps = c_eps * ((Ci - Gamma) / (Ci + 2*Gamma)) # 
        sigma_car = c_car_1 * Ti ** 2 + c_car_2 * Ti + c_car_3 # carboxylation conductance [m/s]
        sigma_CO2 = 1/((1/c_bnd) + (1/c_stm) + (1/sigma_car)) # canopy conductance [m/s]
        
        r_gr = c_r_gr_max * (Dn/(Ds + Dn)) * (c_Q10_gr) ** ((Ti - 20)/10)
        phi_resp = (c_resp_s * (1-c_tau) + c_resp_r * c_tau) * Ds * c_Q10_resp ** ((Ti - 25)/10)
        phi_phot_max = (eps * c_par * c_rad_rf * I_total * sigma_CO2 * (Ci - Gamma)) / (eps * c_par * c_rad_rf * I_total + sigma_CO2 * (Ci - Gamma)) # response of canopy photosynthesis [kg/m^2/s]
        phi_phot = phi_phot_max * (1 - np.exp(-c_k * c_lar_s * (1-c_tau) * Ds)) #

        # 7) CO2 injection & ventilation
        C_inj  = phi_co2_max * u_co2
        C_vent = (Ci - Co) * u_fan_max * u_fan
        C_pho  = phi_phot - (1/c_alpha) * phi_resp - ((1-c_beta)/(c_alpha*c_beta)) * r_gr * Ds # unit change: [umol/m²/s] -> [lg/m³]
        
        
        # 8) ODEs
        dTi_dt = (1 / k_cap_q) * (Q_sol + Q_lamp - Q_cov - Q_trans - Q_vent + Q_heat + Q_cool) # [°C/m^2/s]
        dHi_dt = (1 / k_cap_h) * (H_transp - H_cov - H_vent - H_dehum)                         # [kg/m^3/s]
        dCi_dt = (1 / k_cap_c) * (C_inj - C_vent - C_pho)                                      # [kg/m^3/s]
        dDn_dt = c_alpha * phi_phot - r_gr * Ds - phi_resp - ((1-c_beta)/c_beta) * r_gr * Ds   # [kg/m^2/s]
        dDs_dt = r_gr * Ds # [kg/m^2/s]
        
        
        dxdt = np.array([dTi_dt, dHi_dt, dCi_dt, dDn_dt, dDs_dt])  # + w add process noise
        return dxdt
    
    
    # RK4 integrator for cc_model_gh.
    def rk4_gh(self, t, dt, xk, uk, ok): 
        k1 = self.cc_model_gh(t,          xk,              uk, ok)
        k2 = self.cc_model_gh(t + 0.5*dt, xk + 0.5*dt*k1,  uk, ok)
        k3 = self.cc_model_gh(t + 0.5*dt, xk + 0.5*dt*k2,  uk, ok)
        k4 = self.cc_model_gh(t + dt,     xk + dt*k3,      uk, ok)
        
        k_sum = k1 + 2*k2 + 2*k3 + k4
        
        return k_sum
        
    def T_wb_calculation(self, T_o, RH_o): # based on wet bulb calculator
        T_wb = T_o * np.arctan(0.151977 * np.sqrt(RH_o + 8.313659)) \
                + 0.00391838 * np.sqrt( RH_o ** 3 ) * np.arctan(0.023101 * RH_o) \
                    -np.arctan(RH_o - 1.676331) + np.arctan(T_o + RH_o) -4.686035
        
        return T_wb

In [8]:
import torch
from cvxpylayers.torch import CvxpyLayer
import cvxpy as cp

class DiffQPLayer:
    def __init__(self, params, device="cpu"):
        self.device = device
        self.K = params["K"]
        self.m = 2 * (params["ny"] * params["T_f"]) # both state and input constraints
        
        # Define the QP problem parametrically
        g = cp.Variable((self.K, 1))  # Optimization variable
        f = cp.Parameter((self.K, 1))  # Linear term (from DNN)
        A = cp.Parameter((self.m, self.K))  # Constraint matrix
        b = cp.Parameter((self.m, 1))  # Constraint bounds
        H = 2 * torch.eye(self.K, dtype=torch.float32, device=device).numpy()  # Constant H
        
        objective = cp.Minimize(0.5 * cp.quad_form(g, H) + f.T @ g)
        constraints = [A @ g <= b]
        problem = cp.Problem(objective, constraints)
        
        # Create differentiable layer
        self.qp_layer = CvxpyLayer(problem, parameters=[f, A, b], variables=[g]) # ignore_dpp=True, solver_args=solver_args
    
    def forward(self, f, A, b):
        # Ensure inputs are in the right shape and on the correct device
        f = f.reshape(-1, self.K, 1).to(self.device)  # (batch, K, 1)
        A = A.reshape(-1, self.m, self.K).to(self.device)  # (batch, m, K)
        b = b.reshape(-1, self.m, 1).to(self.device)  # (batch, m, 1)
        
        # Use ignore_dpp=True and set solver options
        solver_args = {
            "max_iters": 5000,  # Increase iterations for better accuracy
            "eps": 1e-8,        # Tighten tolerance
            # "warm_starts": True,
            "verbose": False    # Suppress solver output
        }
        
        # Solve QP and get solution
        g_star, = self.qp_layer(f, A, b, solver_args=solver_args)
        
        return g_star  # (batch, K, 1)
    
    def __call__(self, f, A, b):
        return self.forward(f, A, b)

In [9]:
from scipy.linalg import block_diag

class DeepDeePC:
    def __init__(self, greenhouse_params, params, dnn_model, device="cpu"):
        self.p = greenhouse_params
        self.params = params
        self.dnn = dnn_model.to(device)
        self.diff_qp_layer = DiffQPLayer(params, device=device)
        self.device = device
        
    def compute_stage_cost(self, u, dt):
        # Retrieve parameters
        cc_e = self.p.get_param("cc_e")          # Electricity cost [$ / kWh]
        cc_c = self.p.get_param("cc_c")          # CO2 cost [$ / kg]
        c_COP = self.p.get_param("c_COP")         # Coefficient of performance (unused here)
        c_dehum_eev = self.p.get_param("c_dehum_eev")  # Dehumidification cost [kg / kWh]
        
        P_heat   = self.p.get_param("P_heat")     # [W/m²]
        P_light  = self.p.get_param("P_light")    # [W/m²]
        fan_eff  = self.p.get_param("fan_eff")    # [m^3/s/kW] (unused here)
        A_s  = self.p.get_param("A_s")            # Surface area [m²] (not used in computation below)
        A_p  = self.p.get_param("A_p")            # Cooling pad surface [m²]
        pad_eff = self.p.get_param("pad_eff")   # [m^3/s/kW]
        u_fan_max  = self.p.get_param("u_fan_max")
        u_dehum_max = self.p.get_param("u_dehum_max")
        u_pad_max = self.p.get_param("u_pad_max")
        phi_co2_max = self.p.get_param("phi_co2_max")  # [kg/m²/s]
        
        # Handle single sample (1D) or batch (2D) input uniformly
        is_single = (u.ndim == 1)
        if is_single:
            u = u[None, :]  # Convert to shape (1, n)

        # Check the dimensionality of u and compute accordingly.
        C_heat  = 1E-3 * (dt / 3600) * P_heat  * u[:, 0]                    # Heating cost  [kWh/m²] 
        C_cool  = (A_p / A_s) * (dt / 3600) * u_pad_max * u[:, 1] / fan_eff # Cooling cost  [kWh/m²] 
        C_blind = 0.0 * u[:, 2]                                             # Blind/pad cost (assume zero if not used)
        C_light = 1E-3 * (dt / 3600) * P_light * u[:, 3]                    # Lighting cost [kWh/m²]
        C_fan   = (dt / 3600) * u_fan_max * u[:, 4]                         # Fan cost      [kWh/m²] # C_fan = (dt/3600) / fan_eff * u_fan_max * u[3]
        C_dehum = dt * u_dehum_max * u[:, 5] / c_dehum_eev                  # Dehumification cost [kWh/m²]

        C_elec  = (C_heat + C_cool + C_light + C_fan + C_dehum + C_blind)   # Total electric energy per unit area
        C_co2   = dt * phi_co2_max * u[:, 6]                                # CO2 cost [kg/m²]

        # Sum all cost components
        C_total = cc_e * C_elec + cc_c * C_co2
        return C_total
    
    def build_hankel_matrices(self, x, u):
        T = self.params["T"]
        T_p = self.params["T_p"]
        L = self.params["L"]  # L = T_p + T_f
        K = self.params["K"]
        
        # For simplicity, let's define y := x if x is indeed the greenhouse output.
        y = x  # if x is your measured output. Adjust as needed.

        # Initialize Hankel blocks:
        Up = [] #   Up  in R^(nu*Tp, K)
        Uf = [] #   Uf  in R^(nu*Tf, K)
        Yp = [] #   Yp  in R^(ny*Tp, K)
        Yf = [] #   Yf  in R^(ny*Tf, K)

        # Stack row by row for each time step in the horizon. 
        for col in range(K):
            # A snippet from 'col' to 'col + L - 1'
            u_block = u[col: col + L, :]  # shape (L, nu)
            y_block = y[col: col + L, :]  # shape (L, ny)

            # Past = first Tp rows
            u_past = u_block[:T_p, :]  # shape (Tp, nu)
            y_past = y_block[:T_p, :]  # shape (Tp, ny)

            # Future = last Tf rows
            u_fut = u_block[T_p:, :]   # shape (Tf, nu)
            y_fut = y_block[T_p:, :]   # shape (Tf, ny)

            # Flatten or stack them as columns:
            Up_col = u_past.reshape(-1, 1)  # shape (nu*Tp, 1)
            Uf_col = u_fut.reshape(-1, 1)   # shape (nu*Tf, 1)
            Yp_col = y_past.reshape(-1, 1)  # shape (ny*Tp, 1)
            Yf_col = y_fut.reshape(-1, 1)   # shape (ny*Tf, 1)

            if col == 0:
                Up = Up_col
                Uf = Uf_col
                Yp = Yp_col
                Yf = Yf_col
            else:
                Up = np.hstack([Up, Up_col])
                Uf = np.hstack([Uf, Uf_col])
                Yp = np.hstack([Yp, Yp_col])
                Yf = np.hstack([Yf, Yf_col])

        return Up, Yp, Uf, Yf
    
    def set_hankel_matrices(self, Uf, Yf):
        """A helper to store Uf and Yf for evaluation (if desired)."""
        self.Uf = Uf
        self.Yf = Yf
    
    def _solve_qp_from_dnn(self, input_vec: torch.Tensor, Uf: np.ndarray, Yf: np.ndarray, time_flag):
        # (a) Forward through DNN
        g_unc = self.dnn(input_vec.to(self.device))    # (batch, K)
        batch_size, K = g_unc.shape
        dtype = g_unc.dtype

        # (b) Build f for the QP: f = -2 * g_unc.unsqueeze(-1)
        f = -2 * g_unc.unsqueeze(-1)  # shape=(batch, K, 1)

        # (c) Prepare time_flag as a length‐batch tensor
        if not torch.is_tensor(time_flag):
            time_flag = torch.tensor(time_flag, dtype=dtype, device=self.device).unsqueeze(0)
        time_flag = time_flag.reshape(-1)              # (batch,)
        if time_flag.dim() == 0:
            time_flag = time_flag.unsqueeze(0)

        # (d) Use p.set_min_max() to get day/night x_min, x_max
        x_min_day, x_max_day, x_min_night, x_max_night = self.p.set_min_max()

        x_min_orig = torch.where(
            time_flag[:, None] == 1,
            torch.as_tensor(x_min_day,   dtype=torch.float32, device=self.device),
            torch.as_tensor(x_min_night, dtype=torch.float32, device=self.device),
        )  # shape=(batch, ny)

        x_max_orig = torch.where(
            time_flag[:, None] == 1,
            torch.as_tensor(x_max_day,   dtype=torch.float32, device=self.device),
            torch.as_tensor(x_max_night, dtype=torch.float32, device=self.device),
        )  # shape=(batch, ny)

        # (e) Build Δ = X_max_scale − X_min_scale (for all ny)
        X_min = torch.as_tensor(self.params["x_min_value_scaling"], dtype=torch.float32, device=self.device)
        X_max = torch.as_tensor(self.params["x_max_value_scaling"], dtype=torch.float32, device=self.device)
        delta = X_max - X_min  # (ny,)

        # (f) Build b_state = [ (x_max_orig − X_min) repeated over T_f;  −(x_min_orig − X_min) ]
        T_f = self.params["T_f"]
        #  b_pos shape: (batch, ny*T_f)
        b_pos = (x_max_orig - X_min) \
                    .unsqueeze(2) \
                    .repeat(1, 1, T_f) \
                    .view(batch_size, -1)
        #  b_neg shape: (batch, ny*T_f)
        b_neg = -(x_min_orig - X_min) \
                    .unsqueeze(2) \
                    .repeat(1, 1, T_f) \
                    .view(batch_size, -1)
        #  Concatenate → (batch, 2*ny*T_f) then unsqueeze to (batch, 2*ny*T_f, 1)
        b_state = torch.cat([b_pos, b_neg], dim=1).unsqueeze(-1)

        # (g) Build A_state = [ D·Yf; −D·Yf ], where D = diag(delta repeated T_f times)
        delta_rep = delta.repeat_interleave(T_f)           # (ny*T_f,)
        D = torch.diag(delta_rep).to(self.device)         # (ny*T_f, ny*T_f)

        Yf_torch = torch.as_tensor(Yf, dtype=torch.float32, device=self.device)  # (ny*T_f, K)
        Yf_orig = D @ Yf_torch                             # (ny*T_f, K)

        A_pos = Yf_orig                                     # (ny*T_f, K)
        A_state = torch.cat([A_pos, -A_pos], dim=0)         # (2*ny*T_f, K)
        # Now duplicate along batch dimension:
        A_state_batch = A_state.unsqueeze(0).repeat(batch_size, 1, 1)  # (batch, 2*ny*T_f, K)

        # (h) Solve QP: g_star shape = (batch, K, 1)
        g_star = self.diff_qp_layer(f, A_state_batch, b_state)
        g_star = g_star.squeeze(-1)   # (batch, K)

        # If batch_size=1, return a 1D vector of shape (K,)
        return g_star.squeeze(0) if batch_size == 1 else g_star
    
    def predict_gk(self, input_vec, Uf, Yf, time_flag):
        self.dnn.eval()
        
        # forward pass
        with torch.no_grad():
            pred_g = self.dnn(input_vec)  # Shape: (batch, K)
        
        return pred_g.squeeze(0)
    
    def solve_deep_deepc(self, Up, Yp, Uf, Yf, u_ini, y_ini, e_u, e_y, time_feature, time_flag): # , x_min, x_max
        
        u_ini_tensor = torch.as_tensor(u_ini, dtype=torch.float32).view(1, -1) # shape: (1, nu*T_p)
        y_ini_tensor = torch.as_tensor(y_ini, dtype=torch.float32).view(1, -1) # shape: (1, ny*T_p)
        e_u_tensor = torch.as_tensor(e_u, dtype=torch.float32).view(1, -1) # shape: (1, nu)
        e_y_tensor = torch.as_tensor(e_y, dtype=torch.float32).view(1, -1) # shape: (1, ny)
        time_feature_tensor = torch.as_tensor(time_feature, dtype=torch.float32).view(1, -1)  # shape: (1, T_p+1)
        Uf_tensor = torch.tensor(Uf, dtype=torch.float32, device=self.device)
        
        # Concatenate along the feature dimension
        input_vec = torch.cat([u_ini_tensor, y_ini_tensor, e_u_tensor, e_y_tensor, time_feature_tensor], dim=1) # shape: (1, input_dim)
        
        g_opt = self.predict_gk(input_vec, Uf, Yf, time_flag)  # shape (K,1)
        
        # 2) Reconstruct u_hat, y_hat from Hankel blocks
        u_hat = self.uf_clamp(g_opt, Uf_tensor, time_flag) # torch.tensor(Uf, dtype=torch.float32).to(g_opt.device) @ g_opt  # shape (nu*T_f,1)
        y_hat = torch.tensor(Yf, dtype=torch.float32).to(g_opt.device) @ g_opt  # shape (ny*T_f,1)

        return u_hat, y_hat, g_opt
        
    def train_dnn_for_deepc(self, dnn_model, dataloader, Up, Yp, Uf, Yf, num_epochs=1000, lr=1e-4, 
                            start_epoch=0, optimizer=None, checkpoint_path=None,
                            save_every=100, loss_history=[]):
        if optimizer is None:
            optimizer = torch.optim.Adam(dnn_model.parameters(), lr=lr)
            
        dnn_model.train()
        
        Q_diag = self.create_block_diag(self.params["Q"], self.params["T_f"])
        R_diag = self.create_block_diag(self.params["R"], self.params["T_f"])
        
        # Convert the (offline) Hankel block for the future to torch tensors.
        Uf_tensor = torch.tensor(Uf, dtype=torch.float32).T  # shape: (K, nu*T_f)
        Yf_tensor = torch.tensor(Yf, dtype=torch.float32).T  # shape: (K, ny*T_f)
        
        # Convert the weighting matrices Q and R once to torch tensors.
        Q_tensor = torch.tensor(Q_diag, dtype=torch.float32)  # Q has shape (ny*T_f, ny*T_f)
        R_tensor = torch.tensor(R_diag, dtype=torch.float32)  # R has shape (nu*T_f, nu*T_f)
        
        alpha_u_lb = torch.tensor(self.params["alpha_u_lb"], dtype=torch.float32)  # shape: (nu*T_f, nu*T_f)
        alpha_u_ub = torch.tensor(self.params["alpha_u_ub"], dtype=torch.float32)
        
        P_y_lb = torch.tensor(self.params["P_y_lb"], dtype=torch.float32)   # shape: (ny*T_f,)
        P_y_ub = torch.tensor(self.params["P_y_ub"], dtype=torch.float32)   # same shape
        
        total_epochs = start_epoch + num_epochs
        
        # Create a single tqdm progress bar
        pbar = tqdm(total=total_epochs, desc="Training Epochs", unit="epoch")
        
        for epoch in range(start_epoch, total_epochs):
            epoch_loss = 0.0
            total_samples = 0
            
            for batch_idx, (u_ini, y_ini, e_u, e_y, u_ref, y_ref, time_feature, time_flag) in enumerate(dataloader):
                batch_size = u_ini.shape[0]
                total_samples += batch_size
                                
                # Flatten and concatenate the inputs
                u_ini_flat = u_ini.view(u_ini.size(0), -1)      # (batch, nu * T_p)
                y_ini_flat = y_ini.view(y_ini.size(0), -1)      # (batch, ny * T_p)
                e_u_flat = e_u.view(e_u.size(0), -1)            # (batch, nu)
                e_y_flat = e_y.view(e_y.size(0), -1)            # (batch, ny)
                time_feature_flat = time_feature.view(time_feature.size(0), -1) # (batch, T_p + 1)
                
                input_vec = torch.cat([u_ini_flat, y_ini_flat, e_u_flat, e_y_flat, time_feature_flat], dim=1)
                
                # 0-3. Use LSTM model w/ differentiable QP layer for training
                pred_g = self.predict_gk(input_vec, Uf, Yf, time_flag) # time_flag
                
                # 1. Input tracking loss
                pred_u = torch.matmul(pred_g, Uf_tensor) # (batch, nu * T_f)
                u_ref_flat = u_ref.view(u_ref.size(0), -1)      # (batch, nu * T_f)
                u_error = pred_u - u_ref_flat
                loss_u = torch.sum(u_error * torch.matmul(u_error, R_tensor), dim=1)
                
                
                # 2. Output tracking loss
                pred_y = torch.matmul(pred_g, Yf_tensor) # (batch, ny * T_f)
                y_ref_flat = y_ref.view(y_ref.size(0), -1)      # (batch, ny * T_f)
                y_error = pred_y - y_ref_flat
                loss_y = torch.sum(y_error * torch.matmul(y_error, Q_tensor), dim=1)
                
                # ================================================
                # 3-1. Soft-constraint for input
                
                # daytime / nighttime index
                is_daytime = (time_flag[0].item() == 1) # is_daytime = 1: daytime, is_daytime = 0: nighttime
                
                u_min_tensor = torch.tensor(self.params["u_min"], dtype=torch.float32).unsqueeze(0)  # shape: (1, nu*T_f)
                u_max_tensor = torch.tensor(self.params["u_max"], dtype=torch.float32).unsqueeze(0)
                
                if not is_daytime:
                    u_max_tensor[0, 3] = 0
                
                violation_u_lb = F.relu(u_min_tensor - pred_u) # broadcasting is applied to u_min_tensor, u_max_tensor for batch_size
                violation_u_ub = F.relu(pred_u - u_max_tensor)
                pen_u_lb = alpha_u_lb * (violation_u_lb**2).sum(dim=1)
                pen_u_ub = alpha_u_ub * (violation_u_ub**2).sum(dim=1)
                
                loss_pu = pen_u_lb + pen_u_ub
                # ================================================
                
                # ================================================
                # 3-2. Soft-constraint for state                
                def get_bound(param_day, param_night):
                    try:
                        return self.p.get_param(param_night)
                    except Exception:
                        return self.p.get_param(param_day)
                
                y_min_vals = [
                    self.p.get_param("Ti_min_day") if is_daytime else get_bound("Ti_min_day", "Ti_min_night"),
                    self.p.get_param("Hi_min_day") if is_daytime else get_bound("Hi_min_day", "Hi_min_night"),
                    self.p.get_param("Ci_min_day") if is_daytime else get_bound("Ci_min_day", "Ci_min_night"),
                    self.p.get_param("Dw_min"),
                    self.p.get_param("Dw_min")
                ]
                y_max_vals = [
                    self.p.get_param("Ti_max_day") if is_daytime else get_bound("Ti_max_day", "Ti_max_night"),
                    self.p.get_param("Hi_max_day") if is_daytime else get_bound("Hi_max_day", "Hi_max_night"),
                    self.p.get_param("Ci_max_day") if is_daytime else get_bound("Ci_max_day", "Ci_max_night"),
                    self.p.get_param("Dw_max"),
                    self.p.get_param("Dw_max")
                ]
                
                y_min_vals = np.array(y_min_vals, dtype=np.float32)
                y_max_vals = np.array(y_max_vals, dtype=np.float32)
                
                y_min_vals = self.min_max_scale(y_min_vals, self.params["x_min_value_scaling"], self.params["x_max_value_scaling"])
                y_max_vals = self.min_max_scale(y_max_vals, self.params["x_min_value_scaling"], self.params["x_max_value_scaling"])
                
                y_min_tensor = torch.tensor(y_min_vals, dtype=torch.float32, device=pred_y.device).repeat(1, self.params["T_f"])
                y_max_tensor = torch.tensor(y_max_vals, dtype=torch.float32, device=pred_y.device).repeat(1, self.params["T_f"])
                
                violation_y_lb = F.relu(y_min_tensor - pred_y)
                violation_y_ub = F.relu(pred_y - y_max_tensor)
                # pen_y_lb = alpha_y_lb * (violation_y_lb**2).sum(dim=1)
                # pen_y_ub = alpha_y_ub * (violation_y_ub**2).sum(dim=1)
                
                pen_y_lb = (P_y_lb * violation_y_lb**2).sum(dim=1)
                pen_y_ub = (P_y_ub * violation_y_ub**2).sum(dim=1)
                
                loss_py = pen_y_lb + pen_y_ub
                
                loss_p_total = loss_pu + loss_py
                # ================================================
                
                # ================================================
                # 3-2. Soft-constraint for unscaled variables (unit: [℃], [RH], [ppm])
                pred_y_3d = pred_y.view(batch_size, self.params["T_f"], self.params["ny"])   # e.g. (32, 2, 5)
                
                Xmin = torch.tensor(self.params["x_min_value_scaling"], device=pred_y.device)
                Xmax = torch.tensor(self.params["x_max_value_scaling"], device=pred_y.device)
                Xmin = Xmin.view(1, 1, self.params["ny"])
                Xmax = Xmax.view(1, 1, self.params["ny"])
                
                pred_y_unscale = pred_y_3d * (Xmax - Xmin) + Xmin # unscaling shape: (batch, Tf, ny)
                
                # Select the correct RH bounds:
                if time_flag[0].item() == 1:  # daytime
                    RH_min, RH_max = self.p.get_param("RHi_min_day"), self.p.get_param("RHi_max_day")
                    Ti_min, Ti_max = self.p.get_param("Ti_min_day"), self.p.get_param("Ti_max_day")
                    Ci_min, Ci_max = self.p.get_param("Ci_min_day_ppm"), self.p.get_param("Ci_max_day_ppm")
                else:                        # nighttime
                    RH_min, RH_max = self.p.get_param("RHi_min_night"), self.p.get_param("RHi_max_night")
                    Ti_min, Ti_max = self.p.get_param("Ti_min_night"), self.p.get_param("Ti_max_night")
                    Ci_min, Ci_max = self.p.get_param("Ci_min_night_ppm"), self.p.get_param("Ci_max_night_ppm")
                
                Ti = pred_y_unscale[:, :, 0]                             # [°C]
                Hi = pred_y_unscale[:, :, 1]                             # [kg/m³]
                Ci = pred_y_unscale[:, :, 2]                             # [kg/m³]
                Ci_ppm = Ci * (24.45 / self.p.get_param("m_co2")) * 1E3  # [ppm]
                
                Hi_sat = (1.0272 * Ti - 1.8959) * 1e-3  # [kg/m³]
                RHi    = (Hi / Hi_sat) * 100            # [%]
                
                # temperature violation
                viol_T_lb = F.relu(Ti_min - Ti)                        # (batch, Tf)
                viol_T_ub = F.relu(Ti - Ti_max)
                loss_T    = self.params["lambda_T"] * (viol_T_lb**2 + viol_T_ub**2).sum(dim=1)   # (batch,)
                
                # RH violation
                viol_RH_lb = F.relu(RH_min - RHi)   # (batch, T_f)
                viol_RH_ub = F.relu(RHi  - RH_max)
                loss_RH = self.params["lambda_RH"] * (viol_RH_lb**2 + viol_RH_ub**2).sum(dim=1)  
                
                # CO₂ violation
                viol_C_lb = F.relu(Ci_min - Ci_ppm)
                viol_C_ub = F.relu(Ci_ppm - Ci_max)
                loss_CO2  = self.params["lambda_CO2"] * (viol_C_lb**2 + viol_C_ub**2).sum(dim=1)
                # ================================================
                
                
                # ================================================
                # 4. L2 regularization
                loss_reg = 0.0
                for param in dnn_model.parameters():
                    loss_reg += torch.norm(param, 2) ** 2
                loss_reg = self.params["lambda_reg"] * loss_reg
                # ================================================
                
                # ================================================
                # 5. Energy minimization
                pred_u_reshape = pred_u.view(pred_u.size(0), self.params["T_f"], self.params["nu"])
                
                loss_e = self.compute_stage_cost(pred_u_reshape.reshape(-1, self.params["nu"]), dt=600)
                loss_e = self.params["lambda_e"] * loss_e.view(pred_u.size(0), self.params["T_f"]).mean(dim=1)
                # ================================================
                
                # ================================================
                # 6. Crop yield maximization
                pred_y_reshape = pred_y.view(pred_y.size(0), self.params["T_f"], self.params["ny"])
                yield_value = pred_y_reshape[:, -1, 3] + pred_y_reshape[:, -1, 4]
                loss_c = self.params["lambda_c"] * -yield_value # (batch, )
                # ================================================
                
                
                # ================================================
                # 7. Total loss
                total_loss = torch.mean(loss_y + loss_u + loss_p_total) \
                                + loss_T.mean() \
                                + loss_RH.mean() \
                                + loss_CO2.mean() \
                                + loss_reg \
                                + torch.mean(loss_e) \
                                + torch.mean(loss_c)
                # ================================================
                
                # if batch_idx % 20 == 0:
                #     print(f"Daytime index: {is_daytime}")
                #     print(f"Epoch {epoch}, Batch {batch_idx}:")
                #     print(f"Tracking: y={loss_y.mean().item():.2f}, u={loss_u.mean().item():.2f}, pu={loss_pu.mean().item():.2f}")
                #     print(f"Penalties: T={loss_T.mean().item():.2f}, RH={loss_RH.mean().item():.2f}, CO2={loss_CO2.mean().item():.2f}")
                #     print(f"Other: reg={loss_reg.item():.2f}, e={loss_e.mean().item():.2f}, c={loss_c.mean().item():.2f}")
                #     print(f"Violations: T={viol_T_lb.mean().item():.2f}/{viol_T_ub.mean().item():.2f}, "
                #           f"RH={viol_RH_lb.mean().item():.2f}/{viol_RH_ub.mean().item():.2f}, "
                #           f"CO2={viol_C_lb.mean().item():.2f}/{viol_C_ub.mean().item():.2f}")
                #     print(f"Scaled y: {pred_y[0].detach().cpu().numpy()}")
                #     print(f"Unscaled T_i: {Ti[0,0].item():.2f}, RH_i: {RHi[0,0].item():.2f}, Ci_ppm: {Ci_ppm[0,0].item():.2f}")
                
                
                optimizer.zero_grad()
                total_loss.backward()
                
                # gradient clipping
                # grad_norm = torch.nn.utils.clip_grad_norm_(dnn_model.parameters(), max_norm=1.0) # max_norm=10.0
                
                # if batch_idx % 20 == 0:
                #     print(f"[Debug] grad norm before clipping: {grad_norm:.4f}")
                
                
                optimizer.step()
                epoch_loss += total_loss.item() * batch_size
                
            avg_loss = epoch_loss / total_samples
            loss_history.append(avg_loss)
            
            # Update the progress bar's postfix and increment it by 1 epoch
            pbar.set_postfix(loss=f"{avg_loss:.5f}")
            pbar.update(1)
            
            if checkpoint_path is not None and (epoch + 1) % save_every == 0:
                save_dict = {
                    'epoch': epoch + 1,
                    'model_state_dict': dnn_model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss_history': loss_history,
                }
                torch.save(save_dict, checkpoint_path)
                print(f"Checkpoint saved at epoch {epoch+1} to {checkpoint_path}")
            
        print("Training complete.")
        return dnn_model, loss_history
    
    def get_first_control(self, u_opt):
        # Reshape
        u_opt_2d = u_opt.reshape(self.params["T_f"], self.params["nu"]) # shape (Tf, nu)
        # The first row => the next control
        return u_opt_2d[0, :]
    
    def get_first_state(self, y_opt):
        # Reshape
        y_opt_2d = y_opt.reshape(self.params["T_f"], self.params["ny"])  # shape (Tf, nu)
        # The first row => the next control
        return y_opt_2d[0, :]
    
    def min_max_scale(self, X, X_min, X_max):
        # Avoid division by zero if X_max == X_min. Add small epsilon if needed.
        return (X - X_min) / (X_max - X_min + 1e-12)

    def min_max_unscale(self, X_scaled, X_min, X_max):
        return X_scaled * (X_max - X_min) + X_min

    def evaluate_validation(self, validation_loader):
        if self.Uf is None or self.Yf is None:
            raise ValueError("Hankel matrices Uf and Yf must be set before evaluation. Call set_hankel_matrices().")
        
        # Convert Uf and Yf to torch tensors for reconstruction.
        Uf_tensor = torch.tensor(self.Uf, dtype=torch.float32).T  # shape: (K, nu*T_f)
        Yf_tensor = torch.tensor(self.Yf, dtype=torch.float32).T  # shape: (K, ny*T_f)
        
        self.dnn.eval()
        losses = []
        for (u_ini, y_ini, e_u, e_y, u_ref, y_ref, time_feature) in validation_loader:
            # Flatten and concatenate the inputs.
            u_ini_flat = u_ini.view(u_ini.size(0), -1)
            y_ini_flat = y_ini.view(y_ini.size(0), -1)
            e_u_flat = e_u.view(e_u.size(0), -1)            # shape (batch, nu)
            e_y_flat = e_y.view(e_y.size(0), -1)            # shape (batch, ny)
            u_ref_flat = u_ref.view(u_ref.size(0), -1)
            y_ref_flat = y_ref.view(y_ref.size(0), -1)
            time_feature_flat = time_feature.view(time_feature.size(0), -1)
            
            input_vec = torch.cat([u_ini_flat, y_ini_flat, e_u_flat, e_y_flat, time_feature_flat], dim=1)
            
            with torch.no_grad():
                pred_g = self.dnn(input_vec)  # shape: (batch, K)
            
            # Reconstruct the predicted trajectories.
            pred_u = torch.matmul(pred_g, Uf_tensor)  # shape: (batch, nu*T_f)
            pred_y = torch.matmul(pred_g, Yf_tensor)   # shape: (batch, ny*T_f)
            
            # Compute the error between the predicted and reference trajectories.
            u_error = pred_u - u_ref_flat
            y_error = pred_y - y_ref_flat
            
            # Use a simple mean squared error (MSE) as the performance metric.
            loss_batch = torch.mean(u_error**2 + y_error**2).item()
            # loss_batch = torch.mean(u_error**2).item()
            losses.append(loss_batch)
        
        return np.mean(losses)
    
    def uf_clamp(self, g, Uf_tensor, time_flag):
        u_raw = torch.matmul(g, Uf_tensor.T)                     # (batch, nu*Tf)
        u_min_tensor = torch.as_tensor(self.params["u_min"], dtype=g.dtype, device=g.device)
        u_max_tensor = torch.as_tensor(self.params["u_max"], dtype=g.dtype, device=g.device)
        
        if time_flag == 0: # nighttime
            u_max_tensor[3] = 0   # lighting -> 0
            
        return torch.clamp(u_raw, u_min_tensor, u_max_tensor)                  # (batch, nu*Tf)
    
    
    def create_block_diag(self, base_matrix, horizon):
        return block_diag(*[base_matrix for _ in range(horizon)])

In [10]:
from torch.utils.data import Dataset, DataLoader, random_split

class DeepDeePCDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list
    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        u_ini, y_ini, e_u, e_y, u_ref, y_ref, time_feature, time_flag = self.data_list[idx]
        return (torch.tensor(u_ini, dtype=torch.float32),
                torch.tensor(y_ini, dtype=torch.float32),
                torch.tensor(e_u, dtype=torch.float32),
                torch.tensor(e_y, dtype=torch.float32),
                torch.tensor(u_ref, dtype=torch.float32),
                torch.tensor(y_ref, dtype=torch.float32),
                torch.tensor(time_feature, dtype=torch.float32),
                torch.tensor(time_flag, dtype=torch.float32),
                )

In [11]:
def min_max_scale(X, X_min, X_max):
    # Avoid division by zero if X_max == X_min. Add small epsilon if needed.
    return (X - X_min) / (X_max - X_min + 1e-12)

def min_max_unscale(X_scaled, X_min, X_max):
    return X_scaled * (X_max - X_min) + X_min


In [12]:
class DNNPredictor_LSTM(nn.Module):
    def __init__(self, input_dim, output_dim, lstm_hyperparams, seq_length):
        super(DNNPredictor_LSTM, self).__init__()
        self.seq_length = seq_length
        if input_dim % seq_length != 0:
            raise ValueError("input_dim must be divisible by seq_length")
        self.feature_dim = input_dim // seq_length  # e.g., 70 // 7 = 10
        
        hidden_size = lstm_hyperparams.get("hidden_size")
        num_layers = lstm_hyperparams.get("num_layers")
        dropout_rate = lstm_hyperparams.get("dropout")
        
        # LSTM layer: input shape (batch, seq_length, feature_dim)
        self.lstm = nn.LSTM(input_size=self.feature_dim, hidden_size=hidden_size, 
                            num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        
        # Final linear projection to output_dim
        self.fc_out = nn.Linear(hidden_size, output_dim)
        self.bn_out = nn.BatchNorm1d(output_dim, momentum=0.05) # slower updates
        
    def forward(self, x):
        batch_size = x.size(0)
        # Reshape x to (batch, seq_length, feature_dim)
        x = x.view(batch_size, self.seq_length, self.feature_dim)
        
        # Pass through LSTM; we use only the last hidden state.
        lstm_out, (h_n, c_n) = self.lstm(x)
        # h_n has shape (num_layers, batch, hidden_size). Use the last layer's hidden state.
        h_last = h_n[-1]  # shape (batch, hidden_size)
        
        # Final linear mapping to output dimension.
        out = self.fc_out(h_last)  # shape (batch, output_dim)
        return self.bn_out(out)

In [13]:
import matplotlib.pyplot as plt

class FigurePlotter:
    """
    Compact plotting of states and control inputs for comparison.
    """
    def __init__(self, params):
        self.p = params

    def _to_days(self, t_s):
        return t_s / 3600.0 / 24.0

    def plot_states(self, time_s, x_deepc, x_nmpc, x_min=None, x_max=None,
                    titles=None):
        """
        Plot Deep DeePC vs. NMPC states side by side.
        Optionally include min/max bounds.
        """
        t = self._to_days(time_s)
        n_vars = x_deepc.shape[1]
        titles = titles or [f"Var{i}" for i in range(n_vars)]
        fig, axs = plt.subplots(1, n_vars, figsize=(4 * n_vars, 3), sharex=True)
        for i, ax in enumerate(axs):
            ax.plot(t, x_nmpc[:, i], '--', label='NMPC')
            ax.plot(t, x_deepc[:, i], '-', label='Deep DeePC')
            if x_min is not None and x_max is not None:
                ax.plot(t, x_min[:, i], 'r--')
                ax.plot(t, x_max[:, i], 'r--')
            ax.set(title=titles[i])
            ax.legend(loc='upper left', fontsize='small')
            ax.grid(True)
        plt.tight_layout()
        plt.show()

    def plot_controls(self, time_s, u_deepc, u_nmpc, titles=None):
        """
        Plot step comparison of control inputs.
        """
        t = self._to_days(time_s)
        n_ctrl = u_deepc.shape[1]
        titles = titles or [f"u{i}" for i in range(n_ctrl)]
        fig, axs = plt.subplots(1, n_ctrl, figsize=(3 * n_ctrl, 3), sharex=True)
        for i, ax in enumerate(axs):
            ax.step(t, u_nmpc[:, i], '--', where='post', label='NMPC', alpha=0.5)
            ax.step(t, u_deepc[:, i], '-', where='post', label='Deep DeePC')
            ax.set(title=titles[i])
            ax.legend(loc='upper left', fontsize='small')
            ax.grid(True)
        plt.tight_layout()
        plt.show()

In [14]:
import pickle

def save_data(file_path, data):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)
    print(f"Data successfully saved to {file_path}")
    
def load_data(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    print(f"Data successfully loaded from {file_path}")
    return data

In [15]:
def gen_daytime_index(solar_radiation, window_size=15, Io_low=50.0, Io_high=100.0):
    weights = np.ones(window_size) / window_size
    Io_avg = np.convolve(solar_radiation, weights, mode='valid')
    
    # Pad the array to match original length, filling edges appropriately
    pad_start = (window_size - 1) // 2
    pad_end = window_size - 1 - pad_start
    Io_avg = np.pad(Io_avg, (pad_start, pad_end), mode='edge')
    
    # Binary classification using midpoint threshold
    threshold = (Io_low + Io_high) / 2  # 75.0
    daytime_index = np.where(Io_avg >= threshold, 1.0, 0.0)
    
    # Reshape to (total_steps, 1) as in your original code
    return daytime_index.reshape(-1, 1)

## 2. Simulation Setting

In [ ]:
dataset_days = 30

N = 2
filename = f"./inputs/OSU_{dataset_days}days_N{N}_v2.pkl"
data = load_data(filename)

dt = 600
time_per_day = np.arange(0, 86400 + dt, dt)
num_steps_per_day = len(time_per_day) - 1

total_days = 15 # 5
num_data = num_steps_per_day * total_days

x = data["x"][1:num_data+1,:]  # (T, nx) remove the first element, x0
u = data["u"][:num_data,:]     # (T, nu)
o = data["df_outdoor"].iloc[:num_data, [0, 1, 3]].values.astype(float) # [To, Ho, Io]

daytime_idx = gen_daytime_index(o[:,2], window_size=15, Io_low=50.0, Io_high=100.0)

x_min_value = x.min(axis=0, keepdims=True)  # shape (1, ny)
x_max_value = x.max(axis=0, keepdims=True)

x_scaled = min_max_scale(x, x_min_value, x_max_value)

p = GreenhouseParams()
model = CropClimateModel(p)
plotter = FigurePlotter(p)

Data successfully loaded from ./inputs/OSU_30days_N2_v2.pkl


In [17]:
from scipy.linalg import block_diag

T = len(x)     # total number of dataset
T_p = 7        # past horizon
T_f = 3        # future horizon
L = T_p + T_f  # (past + future) horizon
K = T - L + 1  # number of columns in the Hankel

nx = x.shape[1]
ny = x.shape[1]
nu = u.shape[1]

# Choose a candidate pair (for example, the third candidate in each range)
lambda_Q = 1e1  # 1e0
lambda_R = 1e0 # 1e-1 # 5e0

# Base weighting matrices
# base_Q = np.eye(ny) #
base_Q = np.diag([1e0, 1e0, 1e1, 1e0, 1e0])
base_R = np.eye(nu) # np.diag([1e1, 1e1, 1e0, 2e-1, 1e0, 1e0, 5e-1])
# base_R = np.diag([1e1, 1e0, 2e-1, 1e0, 5e-1]) # 1e2 * np.eye(nu * T_f) # 1e-2

# Set Q and R using the chosen lambda values
Q = lambda_Q * base_Q
R = lambda_R * base_R

# penalty for soft constraint terms
alpha_u_lb = 0e1 # 1e+1 # 5e0
alpha_u_ub = 0e1 # 1e+1
alpha_y_lb = 1e1 # 1e+1
alpha_y_ub = 1e1 # 1e+1

# penalty for relative humidity
# lambda_T = 1e0 # 1e0
# lambda_RH = 3e0
# lambda_CO2 = 5e-3 # 1e1 # 5e-1 # 2e-1 # 5e-2

lambda_T = 1e-2 # 1e0
lambda_RH = 1e-2
lambda_CO2 = 1e-3 # 1e1 # 5e-1 # 2e-1 # 5e-2


# energy cost and crop yield terms
lambda_e = 1e0 # 5e1 # 1e0
lambda_c = 2e-1 # 1e1

P_u_lb = np.eye(nu * T_f) * alpha_u_lb
P_u_ub = np.eye(nu * T_f) * alpha_u_ub
# P_y_lb = np.eye(ny * T_f) * alpha_y_lb
# P_y_ub = np.eye(ny * T_f) * alpha_y_ub
P_y_lb = np.tile([1, 1, 1e1, 1, 1], T_f) * alpha_y_lb # np.eye(ny * T_f) * alpha_y_lb
P_y_ub = np.tile([1, 1, 1e1, 1, 1], T_f) * alpha_y_ub # np.eye(ny * T_f) * alpha_y_lb

u_min = np.zeros(nu * T_f)
u_max = np.ones(nu * T_f)

lambda_reg = 1e-4 # l2 regularization hyperparameter for loss function
k_idx_diffQP = 10 # Hybrid training strategy index

params = {
        "T": T,
        "T_p": T_p,
        "T_f": T_f,
        "L": L,
        "K": K,
        "nx": nx,
        "ny": ny,
        "nu": nu,
        "Q": Q,
        "R": R,
        "lambda_T": lambda_T,
        "lambda_RH": lambda_RH,
        "lambda_CO2": lambda_CO2,
        "lambda_reg": lambda_reg,
        "lambda_e": lambda_e, # weight for energy minimization
        "lambda_c": lambda_c, # weight for crop yield maximization
        "alpha_u_lb": alpha_u_lb,
        "alpha_u_ub": alpha_u_ub,
        "alpha_y_lb": alpha_y_lb,
        "alpha_y_ub": alpha_y_ub,
        "P_y_lb": P_y_lb,
        "P_y_ub": P_y_ub,
        "u_min": u_min,
        "u_max": u_max,
        "x_min_value_scaling": x_min_value,
        "x_max_value_scaling": x_max_value,
        "k_idx_diffQP": k_idx_diffQP,
    }


In [18]:
lstm_hyperparams = {
    "batch_size": 64,  # For example
    "hidden_size": 128,
    "num_layers": 4,
    "dropout": 0.2,
}

input_dim = (nu + ny + 1) * (T_p + 1)  # e.g., 70
output_dim = K                         # e.g., 1417
seq_length = T_p + 1                   # e.g., 7


model_type = "lstm"               # Options: "dnn", "lstm", "bilstm", "gru"
dnn_model = DNNPredictor_LSTM(input_dim, output_dim, lstm_hyperparams, seq_length=seq_length)

deep_deepc = DeepDeePC(p, params, dnn_model)
Up, Yp, Uf, Yf = deep_deepc.build_hankel_matrices(x_scaled, u)

deep_deepc.set_hankel_matrices(Uf, Yf)

/Users/jinsungkim/anaconda3/envs/jskim_deepc_v4/lib/python3.9/site-packages/cvxpy/reductions/solvers/solving_chain.py:254: UserWarning: Your problem has too many parameters for efficient DPP compilation. We suggest setting 'ignore_dpp = True'.
  warnings.warn(


In [19]:
import random
from torch.utils.data import Sampler

class TimeFlagSampler(Sampler):
    def __init__(self, data_source, batch_size):
        self.data_source = data_source
        self.batch_size = batch_size

        self.day_indices = []
        self.night_indices = []
        for idx in range(len(data_source)):
            sample = data_source[idx]
            
            if isinstance(sample[-1], torch.Tensor):
                flag_tensor = sample[-1].view(-1)
                time_flag = int(flag_tensor[0].item())
            else:
                time_flag = int(sample[-1])
            
            if time_flag == 1:
                self.day_indices.append(idx)
            else:
                self.night_indices.append(idx)
        
        # Debug distribution
        print(f"Number of daytime indices: {len(self.day_indices)}")
        print(f"Number of nighttime indices: {len(self.night_indices)}")
        
        self.batches = []

        # Create batches for daytime indices, yielding incomplete batches if needed.
        random.shuffle(self.day_indices)
        for i in range(0, len(self.day_indices), batch_size):
            batch = self.day_indices[i:i+batch_size]
            if len(batch) > 0:
                self.batches.append(batch)

        # Create batches for nighttime indices.
        random.shuffle(self.night_indices)
        for i in range(0, len(self.night_indices), batch_size):
            batch = self.night_indices[i:i+batch_size]
            if len(batch) > 0:
                self.batches.append(batch)
                
        # Shuffle the order of mini-batches so training sees a mix of day and night.
        random.shuffle(self.batches)

    def __iter__(self):
        for batch in self.batches:
            yield batch

    def __len__(self):
        return len(self.batches)


In [20]:
# ==== Offline Training Data Generation ====
# We generate training samples by sliding a window over the dataset.
total_dataset_list = []
x_min_day, x_max_day, x_min_night, x_max_night = p.set_min_max()

# For each time index where full past and future windows exist:
for i in range(T_p, T - T_f):
    u_ini = u[i-T_p : i, :].reshape(-1, 1)
    u_ref = u[i : i+T_f, :].reshape(-1, 1)
    y_ini = x_scaled[i-T_p+1 : i+1, :].reshape(-1, 1)
    y_ref = x_scaled[i+1 : i+T_f+1, :].reshape(-1, 1)
    
    e_u = u_ref[0:nu] - u_ini[-nu:]
    e_y = y_ref[0:ny] - y_ini[-ny:]
     
    time_feature = daytime_idx[i-T_p:i+1].reshape(-1, 1)
    time_flag = np.array([int(time_feature[-1, 0])])

    # The DeePC references are taken as u_ref and y_ref.
    total_dataset_list.append((u_ini, y_ini, e_u, e_y, u_ref, y_ref, time_feature, time_flag))

total_dataset = DeepDeePCDataset(total_dataset_list)
total_size = len(total_dataset_list)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

train_dataset, val_dataset = random_split(total_dataset, [train_size, val_size])
print(f"Total # of dataset: {total_size} where # of training: {train_size}, # of validation: {val_size}")

train_loader = DataLoader(train_dataset, batch_sampler=TimeFlagSampler(train_dataset, lstm_hyperparams["batch_size"])) # shuffle = True
val_loader   = DataLoader(val_dataset,   batch_size=lstm_hyperparams["batch_size"], shuffle=False)


Total # of dataset: 422 where # of training: 337, # of validation: 85
Number of daytime indices: 182
Number of nighttime indices: 155


In [ ]:
checkpoint_path = "./checkpoint/lstm.pth"
optimizer = torch.optim.Adam(dnn_model.parameters(), lr=1e-4)

num_training_epochs = 2000
total_epoch = num_training_epochs
# dnn_model, loss_history = deep_deepc.train_dnn_for_deepc(dnn_model, train_loader, Up, Yp, Uf, Yf, num_epochs=num_training_epochs, lr=5e-4, start_epoch=0)
dnn_model, loss_history = deep_deepc.train_dnn_for_deepc(
                            dnn_model, train_loader, Up, Yp, Uf, Yf, 
                            num_epochs=num_training_epochs, lr=1e-4, start_epoch=0,
                            checkpoint_path=checkpoint_path, save_every=200
                            ) # optimizer, loss_history

dnn_model.eval()

## 3. Online Implementation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deep_deepc = DeepDeePC(p, params, dnn_model, device=device)

In [ ]:
import time

# Initialize matrices for states, controls, and cost function over all days
y_total = np.zeros((total_days * num_steps_per_day + (T_p + 1), ny))
y_model = np.zeros((total_days * num_steps_per_day + (T_p + 1), ny))
u_total = np.zeros((total_days * num_steps_per_day + T_p, nu))
g_total = np.zeros((total_days * num_steps_per_day + T_p, K))

# Initialize min / max / reference values
x_min_total = np.zeros((total_days * num_steps_per_day + 1, nx))
x_max_total = np.zeros((total_days * num_steps_per_day + 1, nx))
x_ref_total = np.zeros((total_days * num_steps_per_day + 1, nx))

# For the very first step, assume nighttime references
x_min_total[0, :] = x_min_night
x_max_total[0, :] = x_max_night
x_ref_total[0, :] = 0.5 * (x_min_night + x_max_night)

J = np.zeros(total_days * num_steps_per_day)  # cost function per time steps

# Set the initial condition
# x0, u0 = p.set_initial()
y_total[0, :] = x[T_p-1,:]
y_model[0, :] = x[T_p-1,:]

step_time = np.zeros(total_days * num_steps_per_day)
day_time = np.zeros(total_days)

# Moving average for solar radiation smoothing
window_size = 15
solar_window = np.zeros(window_size)
window_index = 0

Io_low = 50.0     # Nighttime threshold [W/m²]
Io_high = 100.0   # Daytime threshold [W/m²]
alpha = 0.0  # last-step interpolation weight
ramp_steps = 10   # number of steps over which to go from nighttime to daytime

# Start Simulation (Deep DeePC Online Implementation using trained DNN via offline)
print("Starting online control simulation...")

# for day in range(1, 2):
for day in range(1, total_days + 1):
    day_start_time = time.time()
    start_index = (day - 1) * num_steps_per_day
      
    # for k in range(1, T_p+2):
    for k in range(1, num_steps_per_day + 1):
        step_start_time = time.time()
        global_index = start_index + (k-1) + T_p # T_p, T_p + 1, T_p + 2, ...
        
        # Define the constraints based on daytime / nighttime
        ok = data["df_outdoor"].iloc[global_index, [0, 1, 3]].values.astype(float)  # or however you get outside [To, Ho, Io]
        
        # Smooth solar radiation (Io) with moving average
        solar_window[window_index] = ok[-1]  # Io
        window_index = (window_index + 1) % window_size
        Io_avg = np.mean(solar_window)
        
        if Io_avg <= Io_low:
            alpha_target = 0.0  # Night
        elif Io_avg >= Io_high:
            alpha_target = 1.0  # Day
        else:
            alpha_target = (Io_avg - Io_low) / (Io_high - Io_low)  # Transition
        
        # 2. march alpha toward alpha_target by exactly 1/ramp_steps per step
        step = 1.0 / ramp_steps
        if alpha < alpha_target:
            alpha = min(alpha + step, alpha_target)
        elif alpha > alpha_target:
            alpha = max(alpha - step, alpha_target)
        # if alpha == alpha_target, it stays there
        
        # Interpolate state constraints and reference:
        x_min = (1 - alpha) * x_min_night + alpha * x_min_day
        x_max = (1 - alpha) * x_max_night + alpha * x_max_day
        x_ref = 0.5 * (x_min + x_max)
        x_ref[1] = (x_ref[1] / 100) * (1.0272 * x_ref[0] - 1.8959) * 1E-3
        
        # Reference values
        u_ref = u[global_index : global_index + T_f, :].reshape(-1, 1)
        y_ref = x_scaled[global_index + 1 : global_index + T_f + 1, :].reshape(-1, 1)
        time_feature = daytime_idx[global_index-T_p:global_index+1].reshape(-1, 1)
        time_flag = int(time_feature[-1, 0]) # Extract time_flag from the last element of time_feature
        
        # Initial & error values for DNN input
        if k <= T_p:
            u_ini = u[global_index - T_p : global_index, :].reshape(-1, 1)  # shape (nu*T_p,1)
            y_ini = x_scaled[global_index - T_p + 1 : global_index + 1, :].reshape(-1, 1)  # shape (ny*T_p,1)
            e_u = u_ref[0:nu] - u_ini[-nu:]
            e_y = y_ref[0:ny] - y_ini[-ny:]
        else:
            u_ini = u[global_index - T_p : global_index, :].reshape(-1, 1)  # shape (nu*T_p,1)
            y_ini = x_scaled[global_index - T_p : global_index, :].reshape(-1, 1)  # shape (ny*T_p,1)
            
            e_u = u_ref[0:nu] - u_ini[-nu:]
            e_y = y_ref[0:ny] - y_ini[-ny:]
            
            # u_ini = u_total[global_index - 2 * T_p : global_index - T_p, :].reshape(-1, 1)  # shape (nu*T_p,1)
            # y_ini = min_max_scale(y_model[global_index - 2 * T_p : global_index - T_p, :], x_min_value, x_max_value).squeeze().reshape(-1, 1)
        
        
        # Determine the optimal solution via DeePC
        u_opt, y_opt_scaled, g_opt = deep_deepc.solve_deep_deepc(Up, Yp, Uf, Yf, u_ini, y_ini, e_u, e_y, time_feature, time_flag)
        
        # y_opt = min_max_unscale(y_opt_scaled.reshape(-1, ny), x_min_value, x_max_value)
        y_opt = min_max_unscale(y_opt_scaled.detach().reshape(-1, ny), x_min_value, x_max_value)
        
        u_total[global_index - T_p, :] = deep_deepc.get_first_control(u_opt.detach().cpu().numpy()) # u_opt[:nu,].T # Store control input to the control array
        y_total[global_index - T_p + 1, :] = deep_deepc.get_first_state(y_opt.detach().cpu().numpy()) # y_opt[:ny,].T # Store control input to the control array
        g_total[global_index - T_p, :] = g_opt.detach().cpu().numpy().T
        
        # Store min/max constraints for reference
        x_min_total[global_index - T_p + 1, :] = x_min
        x_max_total[global_index - T_p + 1, :] = x_max
        x_ref_total[global_index - T_p + 1, :] = x_ref
        
        
        # 4. Implement (xk, uk, ok) to the greenhouse model
        # xk = min_max_unscale(xk_scaled, x_min_value, x_max_value).squeeze()
        xk = y_model[global_index - T_p, :] # x[T_p-1,:] (in real, x[T_p]), ...
        uk = u_total[global_index - T_p, :] # u[T_p]
        ok = data["df_outdoor"].iloc[global_index, [0, 1, 3]].values.astype(float)  # or however you get outside [To, Ho, Io]
        
        dxdt = model.rk4_gh(time_per_day[k], dt, xk, uk, ok)
        x_next = xk + (dt / 6.0) * dxdt # closed-form results
        
        y_model[global_index - T_p + 1, :] = x_next
        J[global_index - T_p] = deep_deepc.compute_stage_cost(uk, dt) # compute stage cost
        
        # Time for the time step
        step_time[k-1] = time.time() - step_start_time
        print(f"--- DeePC Computation time for each k={k} of Day {day} completed in {step_time[k-1]:.4f} [s] ---")

    # Time for the full day
    day_time[day-1] = time.time() - day_start_time
    print(f"--- Day {day} completed in {day_time[day-1]:.2f} [s] ---")

print(f"------ Average computation time per step: {(np.mean(day_time)/num_steps_per_day):.3f} [s] ------")

## 4. Plotting

In [ ]:
# 1) Build the full-time vector
total_time_s = np.arange(0, total_days*86400 + dt, dt)  # shape (total_days*num_steps_per_day+1,)

# 3) Plot the state variables
# We assume x, x_min_total, x_max_total are the final arrays after simulation
plotter.plot_states_compare(time_s=total_time_s,
                            x_deepc=y_model[:len(total_time_s)], # y_total,
                            x_nmpc=x[T_p-1:len(total_time_s)+T_p-1,:],
                            x_min = x_min_total,
                            x_max = x_max_total,
                            title_str="Greenhouse Internal States (Deep DeePC vs NMPC)")

plotter.plot_controls_compare(time_s=total_time_s[1:],
                            u_deepc=u_total[:len(total_time_s)-1],
                            u_nmpc=u[T_p:len(total_time_s)+T_p-1,:],
                            title_str="Control Input Comparison (Deep DeePC vs NMPC)")


In [ ]:
J_acc = plotter.plot_costs_compare(time_s=total_time_s[1:],
                   J_deepc=J,
                   J_nmpc=data["J"][T_p:len(J)+T_p],
                   title_str="Costs Over Time")

C_heat, C_cool, C_light, C_fan, C_dehum, C_co2 = plotter.plot_component_costs(time_s=total_time_s[1:], u=u_total[:len(total_time_s)-1], dt=dt, title_str="Control Component Costs Over Time") 

J_nmpc = data["J"][T_p:len(J)+T_p]
J_nmpc_acc = np.sum(J_nmpc) * p.get_param("A_s")
Dw_nmpc = x[len(total_time_s)+T_p-2,-1] + x[len(total_time_s)+T_p-2,-2]

print(f"--- [NMPC] Total Cost: {J_nmpc_acc:.2f} [$], Total crop dry weight: {Dw_nmpc:.6f} [kg/m²]")
print(f"--- [Deep DeePC] Total Cost: {J_acc[-1]:.2f} [$], Total crop dry weight: {(y_model[len(total_time_s)-1,-2] + y_model[len(total_time_s)-1,-1]):.6f} [kg/m²]")